[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zhujisheng/learn_python/blob/master/10.音乐/4.MIDI音乐.ipynb)

[《Python应用实战》视频课程](https://study.163.com/course/courseMain.htm?courseId=1209533804&share=2&shareId=400000000624093)

# MIDI音乐

难度：★★★☆☆

- MIDI（Musical Instrument Digital Interface）是一个通讯协议
- MIDI传输每个音符的音高、强度等信息
- 电子乐器发出MIDI消息，播放器播放对应声音
- MIDI格式也可以保存为文件

## mido库

[mido库](https://mido.readthedocs.io/en/latest/index.html)用于处理MIDI消息，支持MIDO消息的传输

#### 安装

`pip install mido`

#### Ports

- 在mido库中，port指发送和接收MIDI消息的实体，可以是MIDI文件、设备、网络接口...
- 要使用port，必须安装一个Backend
- 推荐使用的Backend是[RtMidi](https://pypi.org/project/python-rtmidi/)

  `pip install python-rtmidi`


## MIDI消息与播放

打开缺省的播放设备

In [ ]:
import time
import librosa
import mido

output = mido.open_output()

播放音符A4

In [ ]:
# midi_note = 69
midi_note = librosa.note_to_midi('A4')
m = mido.Message('note_on', note=midi_note, velocity=100)
output.send(m)

time.sleep(1)

# 停止音符A4
m = mido.Message('note_off', note=midi_note, velocity=100)
output.send(m)

## 制作一段音乐

In [ ]:
# 音高、力度、时长(秒)
messages=[('f4', 100, 0.25),
          ('g4', 64, 0.25),
          ('a4', 64, 0.5),
          ('g4', 64, 0.5),
          ('g4', 64, 0.5),
          ('f4', 100, 0.5),
          ('e4', 64, 0.5),
          ('f4', 64, 1.5),
         ]

In [ ]:
# 播放
for message in messages:
    midi_note = librosa.note_to_midi(message[0])
    m = mido.Message('note_on', note=midi_note, velocity=message[1])
    output.send(m)
    time.sleep(message[2])
    m = mido.Message('note_off', note=midi_note, velocity=message[1])
    output.send(m)

## 更改播放乐器

In [ ]:
# 1 钢琴
# 26吉他
# 41 小提琴
m=mido.Message('program_change', program=41)
output.send(m)

## 写入MIDI文件

In [ ]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile()
track = MidiTrack()
mid.tracks.append(track)

# 更改成小提琴声音
m=mido.Message('program_change', program=41)
track.append(m)

for message in messages:
    midi_note = librosa.note_to_midi(message[0])
    m = Message('note_on', note=midi_note, velocity=message[1], time=0)
    track.append(m)
    ticks = mido.second2tick(message[2], mid.ticks_per_beat, 500000)
    m = Message('note_off', note=midi_note, velocity=message[1], time=int(ticks))
    track.append(m)

mid.save('audio/mysong.mid')

## 读入与播放MIDI文件

- 自己控制时间

In [ ]:
for msg in MidiFile('audio/mysong.mid'):
    print(msg)
    time.sleep(msg.time)
    output.send(msg)

- 使用play函数控制时间

In [ ]:
for message in MidiFile('audio/mysong.mid').play():
    print(message)
    output.send(message)

In [ ]:
for message in MidiFile('audio/piano.mid').play():
    print(message)
    output.send(message)